<a href="https://colab.research.google.com/github/pushkal1234/Apache_Airflow/blob/main/Apache_Airflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
virtualenv venv 
source venv/bin/activate #Activate virtual environment
pip install apache-airflow

SyntaxError: ignored

In [ ]:
(venv) $ pip install apache-airflow[postgres,gcp]

In [ ]:
(venv) $ export AIRFLOW_HOME=airflow_home/
(venv) $ airflow initdb

In [ ]:
dags_folder = /path/to/airflow_home/dags
# The executor class that airflow should use. Choices include
# SequentialExecutor, LocalExecutor, CeleryExecutor, DaskExecutor, KubernetesExecutor
executor = LocalExecutor
sql_alchemy_conn = mysql+pymysql://<username>:<password>@localhost:3306/<database name>
# Whether to load the examples that ship with Airflow. It's good to
# get started, but you probably want to set this to False in a production
# environment
load_examples = False

In [ ]:
import feedparser
from airflow import DAG
# Operators; we need this to operate!
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator
from airflow.utils.dates import days_ago
from textblob import TextBlob as tb
import re

In [ ]:
# These args will get passed on to each operator
# You can override them on a per-task basis during operator initialization
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': days_ago(2),
    'email': ['airflow@example.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=1),
    # 'queue': 'bash_queue',
    # 'pool': 'backfill',
    # 'priority_weight': 10,
    # 'end_date': datetime(2016, 1, 1),
    # 'wait_for_downstream': False,
    # 'dag': dag,
    # 'sla': timedelta(hours=2),
    # 'execution_timeout': timedelta(seconds=300),
    # 'on_failure_callback': some_function,
    # 'on_success_callback': some_other_function,
    # 'on_retry_callback': another_function,
    # 'sla_miss_callback': yet_another_function,
    # 'trigger_rule': 'all_success'
}

In [ ]:
def rss_feed_scraper(url):
    feed = feedparser.parse(url)

    # number of stories
    numStories = len(feed['entries'])

    # list to contain polarity of stories
    final = []

    for i in range(0, numStories):
        # print(feed['entries'][i])
        # initial description
        descInit = feed['entries'][i]['summary_detail']['value']
        # cleaning out the img tag
        # descClean = re.sub('\<img.*$', '', descInit)
        cleanr = re.compile('<.*?>')
        descClean = re.sub(cleanr, '', descInit)
        # final description
        desc = tb(descClean)
        print("Description >>>>>>>>>>>>>>>>>>>>")
        print(desc)
        # title of the entry
        title = tb(feed['entries'][i]['title'])
        # final string which contains description and headline to get a better polarity result
        if not title.ends_with("."):
            title = title + "."
        completeString = title + " " + desc
        print("Complete String >>>>>>>>>>>>>>>>")
        print(completeString)
        # appending story headline and descrition polarity to final list
        final.append(completeString.sentiment.polarity)

    # polarity calculations
    finalPolarity = sum(final) / len(final)
    print(min(final))
    worstPolarity = final.index(min(final))
    print(max(final))
    bestPolarity = final.index(max(final))

    print(finalPolarity)
    print(worstPolarity)
    print(bestPolarity)

In [ ]:
dag = DAG(
    'news_fetch',
    default_args=default_args,
    description='News Fetch DAG',
    schedule_interval=timedelta(minutes=1),
)

In [ ]:
t1 = PythonOperator(
    task_id='fox_news_parser',
    depends_on_past=False,
    python_callable=rss_feed_scraper,
    op_kwargs={'url': "http://rss.cnn.com/rss/cnn_topstories.rss"},
    # retries=3,
    dag=dag,
)

t2 = PythonOperator(
    task_id='cnn_news_parser',
    depends_on_past=False,
    python_callable=rss_feed_scraper,
    op_kwargs={'url': "http://feeds.foxnews.com/foxnews/latest"},
    # retries=3,
    dag=dag,
)

In [ ]:
(venv) $ airflow webserver

In [ ]:
(venv) $ export AIRFLOW_HOME=airflow_home/
(venv) $ airflow scheduler